In [35]:
import pandas as pd

In [47]:
PATH_TO_FILE = './Data/corpus2.json'
data = pd.read_json(PATH_TO_FILE)

In [48]:
data.head(5)

,id,cve,project_name,description,mistakes,tag_list
0,1037,CVE-2014-7948,chromium,Google Chrome caches pages despite having an S...,I think that the vulenrability is interesting ...,"[{'id': 8, 'tag': 'bounty'}, {'id': 95, 'tag':..."
1,1158,CVE-2015-6757,chromium,Trying to access data in ServiceWorker after i...,The mistake made here was a basic C coding mis...,"[{'id': 5, 'tag': 'lifetime-1y2y'}, {'id': 8, ..."
2,1489,CVE-2017-5125,chromium,Embargoed. Heap overflow in Skia.\n,None,"[{'id': 14, 'tag': 'cwe-119'}, {'id': 357, 'ta..."
3,184,CVE-2010-4488,chromium,This bug involves an unhandled case in which t...,"This could be a coding or design error, there ...","[{'id': 3, 'tag': 'lifetime-90d180d'}, {'id': ..."
4,1269,CVE-2016-1682,chromium,Checks to verify the content security policy (...,It seems as though the developer just didn't t...,"[{'id': 1, 'tag': 'lifetime-30d'}, {'id': 8, '..."


In [54]:
CVE_description_mistake = data[['cve','description','mistakes']].copy()

In [55]:
CVE_description_mistake

,cve,description,mistakes
0,CVE-2014-7948,Google Chrome caches pages despite having an S...,I think that the vulenrability is interesting ...
1,CVE-2015-6757,Trying to access data in ServiceWorker after i...,The mistake made here was a basic C coding mis...
2,CVE-2017-5125,Embargoed. Heap overflow in Skia.\n,None
3,CVE-2010-4488,This bug involves an unhandled case in which t...,"This could be a coding or design error, there ..."
4,CVE-2016-1682,Checks to verify the content security policy (...,It seems as though the developer just didn't t...
...,...,...,...
721,CVE-2014-1715,Path traversal vulnerability that can lead to ...,"Ultimately, this vunerability was the direct c..."
722,CVE-2014-0116,A fix applied in a previous update didn't acco...,The correct sanitization regex pattern was pre...
723,CVE-2017-5055,Use after free in printing. When the cross pro...,PrintPreviewDone() got called multiple times a...
724,CVE-2010-0656,"Webkit, if given a false directory in the URL ...",The origin of the bug is not really a commit o...


In [56]:
from fuzzywuzzy import process,fuzz

/Users/prajwalkrishn/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [58]:
for col in CVE_description_mistake[['cve','description','mistakes']]:
    CVE_description_mistake[col] = CVE_description_mistake[col].str.strip()
    print("number of unique values in " + str(col) +": "+ str(CVE_description_mistake[col].nunique()))

number of unique values in cve: 725
number of unique values in description: 707
number of unique values in mistakes: 569


In [59]:
unique_cves = CVE_description_mistake['cve'].unique().tolist()
sorted(unique_cves)[:100]

['CVE-2002-0392',
 'CVE-2002-0661',
 'CVE-2002-0840',
 'CVE-2002-1156',
 'CVE-2002-1593',
 'CVE-2003-0020',
 'CVE-2003-0083',
 'CVE-2003-0253',
 'CVE-2004-0113',
 'CVE-2004-0174',
 'CVE-2004-0488',
 'CVE-2004-0493',
 'CVE-2004-0747',
 'CVE-2004-0751',
 'CVE-2004-0809',
 'CVE-2004-0811',
 'CVE-2004-0885',
 'CVE-2004-0942',
 'CVE-2004-1834',
 'CVE-2005-1268',
 'CVE-2005-2088',
 'CVE-2005-2700',
 'CVE-2005-2728',
 'CVE-2005-2970',
 'CVE-2005-3352',
 'CVE-2005-3357',
 'CVE-2006-3747',
 'CVE-2006-5752',
 'CVE-2007-1862',
 'CVE-2007-3847',
 'CVE-2007-5000',
 'CVE-2007-6388',
 'CVE-2007-6420',
 'CVE-2007-6422',
 'CVE-2008-0005',
 'CVE-2008-0456',
 'CVE-2008-2364',
 'CVE-2008-2939',
 'CVE-2008-6998',
 'CVE-2009-1191',
 'CVE-2009-1195',
 'CVE-2009-1412',
 'CVE-2009-1413',
 'CVE-2009-1414',
 'CVE-2009-1441',
 'CVE-2009-2060',
 'CVE-2009-2693',
 'CVE-2009-2901',
 'CVE-2009-2902',
 'CVE-2009-3095',
 'CVE-2009-3263',
 'CVE-2009-3548',
 'CVE-2009-3555',
 'CVE-2009-3934',
 'CVE-2010-0425',
 'CVE-2010

### This below process des not mean anything as the Fuzyy token sort ratio scorer tokenizes the strings and cleans them by returning these strings to lower cases, removing punctuations, and then sorting them alphabetically. After that, it finds the Levenshtein distance and returns the similarity percentage.

In [60]:
process.extract('CVE-2002-0392', unique_cves, scorer=fuzz.token_sort_ratio)

[('CVE-2002-0392', 100),
 ('CVE-2004-0942', 85),
 ('CVE-2012-0031', 77),
 ('CVE-2013-2900', 77),
 ('CVE-2004-0809', 77)]

In [61]:
process.extract('CVE-2010-3411', unique_cves, scorer=fuzz.token_sort_ratio)

[('CVE-2010-3411', 100),
 ('CVE-2010-3111', 92),
 ('CVE-2010-3415', 92),
 ('CVE-2010-3413', 92),
 ('CVE-2010-3112', 92)]

In [62]:
unique_desc = CVE_description_mistake['description'].unique().tolist()
sorted(unique_desc)[:100]

['(Linux Only) A malicious user could remotely construct a relatively long string.\nUpon creating the string, the user could then use it with the javascript alert \nfunction, or use it with an input that could be displayed in an alert. As a result, \nthe display server could crash and lock up the open chrome windows on the target device, \nrequiring a reboot (Denial of Service).',
 'A CGI (Common Gateway Interface) is a protocol used by web servers that runs command-line interface scripts in response to client requests. Apache HTTP Server has a module "mod_cgid" which is responsible for running CGI scripts.\nCVE-2014-0231 was a vulnerability in Apache HTTP Server\'s mod_cgid module where users could create a denial of service attack by causing the process which ran these scripts to hang indefinitely. mod_cgid did not have any timeout feature. If an attacker found a request that caused one of a server\'s CGI scripts hosted in mod_cgid to halt, this would deny service to other users of t

In [63]:
process.extract('(Linux Only) A malicious user could remotely construct a relatively long string.\nUpon creating the string, the user could then use it with the javascript alert \nfunction, or use it with an input that could be displayed in an alert. As a result, \nthe display server could crash and lock up the open chrome windows on the target device, \nrequiring a reboot (Denial of Service).', unique_desc, scorer=fuzz.token_sort_ratio)

[('(Linux Only) A malicious user could remotely construct a relatively long string.\nUpon creating the string, the user could then use it with the javascript alert \nfunction, or use it with an input that could be displayed in an alert. As a result, \nthe display server could crash and lock up the open chrome windows on the target device, \nrequiring a reboot (Denial of Service).',
  100),
 ('A remote attacker could write malicious input to a data stream that would cause\nthe server to crash. Once the server is crashed, subsequently valid requests will continue to crash\nthe server.',
  47),
 ("The multipart stream area of the system doesn't check the length of a boundary string. This\nmeans that a user can potentially use an extremely long boundary string to cause a denial of\nservice attack.",
  45),
 ('When displaying a type of graphic known as an SVG, a value of infinity could\nbe used as one of the attributes, causing a write to an invalid memory location.\nThis would result in a 

In [64]:
#Create tuples of brand names, matched brand names, and the score
score_sort = [(x,) + i
             for x in unique_desc 
             for i in process.extract(x, unique_desc, scorer=fuzz.token_sort_ratio)]
#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['desc_sort','match_sort','score_sort'])
similarity_sort.head()


,desc_sort,match_sort,score_sort
0,Google Chrome caches pages despite having an S...,Google Chrome caches pages despite having an S...,100
1,Google Chrome caches pages despite having an S...,HTTP piplining is not handled properly in this...,39
2,Google Chrome caches pages despite having an S...,A remote attacker could write malicious input ...,38
3,Google Chrome caches pages despite having an S...,Heap use after memory freed in Chrome content ...,37
4,Google Chrome caches pages despite having an S...,"Part of Google's 2D graphic library, Skia, doe...",37


In [67]:
print(len(similarity_sort))

3535


In [69]:
similarity_sort['sorted_desc_sort'] = np.maximum(similarity_sort['desc_sort'], similarity_sort['match_sort'])
similarity_sort.head()

,desc_sort,match_sort,score_sort,sorted_desc_sort
0,Google Chrome caches pages despite having an S...,Google Chrome caches pages despite having an S...,100,Google Chrome caches pages despite having an S...
1,Google Chrome caches pages despite having an S...,HTTP piplining is not handled properly in this...,39,HTTP piplining is not handled properly in this...
2,Google Chrome caches pages despite having an S...,A remote attacker could write malicious input ...,38,Google Chrome caches pages despite having an S...
3,Google Chrome caches pages despite having an S...,Heap use after memory freed in Chrome content ...,37,Heap use after memory freed in Chrome content ...
4,Google Chrome caches pages despite having an S...,"Part of Google's 2D graphic library, Skia, doe...",37,"Part of Google's 2D graphic library, Skia, doe..."


In [73]:
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 75) &
                (similarity_sort['desc_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_desc_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_desc_sort',axis=1).copy()

In [72]:
high_score_sort

,desc_sort,match_sort,score_sort
11,Embargoed. Heap overflow in Skia.,Embargoed. Heap buffer overflow in Skia.,90
47,Out of bounds read in V8.,Embargoed. Out of bounds read in V8.,83
57,Embargoed. URL Spoofing in Omnibox.,Embargoed. URL Spoof in Omnibox.,95
101,Embargoed. Heap overflow in libxml2.,Embargoed. Heap overflow in Skia.,80
151,Embargoed. Use after free in WebAudio.,Embargoed. Use after free in V8.,85
196,URL spoofing in Omnibox.,Embargoed. URL Spoofing in Omnibox.,82
197,URL spoofing in Omnibox.,Embargoed. URL spoofing in Omnibox.,82
339,Embargoed. Use after free in Blink.,Embargoed. Use after free in Apps Bluetooth.,80
341,Heap overflow in Skia.,Heap buffer overflow in Skia.,86
342,Heap overflow in Skia.,Embargoed. Heap overflow in Skia.,81


In [75]:
high_score_sort.groupby(['desc_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

,,match_sort
desc_sort,score_sort,
Embargoed. URL spoofing in Omnibox.,100,Embargoed. URL Spoofing in Omnibox.
"This vulnerability may overload the server process with a specially crafted\nURL. If a special URL is put in a form field and validated using Struts' built\nin URLValidator, the validation may cause a DoS. The URL validation is done\nusing regular expressions. This vulnerability affects Struts versions 2.5 to\n2.5.5. Checkout CVE-2017-9804 and CVE-2016-7672 for similar CVEs.",97,This vulnerability may overload the server pro...
"Uninitialized use in Skia. Uninitialized value was created by an allocation\nof ""src"" in the stack frame of the ""ReadBuffer"" function.",96,Uninitialized use in Skia. Uninitialized value...
Embargoed. URL Spoofing in Omnibox.,95,Embargoed. URL Spoof in Omnibox.
Embargoed. URL spoofing in Omnibox.,95,Embargoed. URL Spoof in Omnibox.
...,...,...
"URL spoofing in OmniBox. Spoofing occurs when inserting ""U+0650"" into\nthe URL.",75,URL spoofing in OmniBox. Similar to CVE-2017-5...
Use after free in ANGLE.,75,Embargoed. Use after free in Blink.
Out of bounds memory access in V8.,75,Embargoed. Out-of-bounds access in V8.
